In [92]:
import Pkg
Pkg.add("Oscar")
Pkg.add("Polymake")
Pkg.add("Combinatorics")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [93]:
using Oscar
using Polymake
using Combinatorics

In [94]:
tv = fulton.NormalToricVariety(INPUT_RAYS=[0 1; 1 0; 0 -1; -2 1]; INPUT_CONES=[[0,1],[1,2],[2,3],[0,3]])

type: NormalToricVariety

INPUT_CONES
	{0 1}
	{1 2}
	{2 3}
	{0 3}

INPUT_RAYS
	0 1
	1 0
	0 -1
	-2 1

In [5]:
tv.SMOOTH

false

In [8]:
tv.SIMPLICIAL

true

In [9]:
tv.RAYS

pm::Matrix<pm::Rational>
0 1
1 0
0 -1
-1 1/2


In [10]:
tv.MAXIMAL_CONES

pm::IncidenceMatrix<pm::NonSymmetric>
{0 1}
{1 2}
{2 3}
{0 3}


In [11]:
# Get the rays of maximal cone i (tv.MAXIMAL_CONES[i,:] will get the incidence matrix booleans)
# findall gets the indices of the "true"s, then we take those specific rows
i = 3
coneRays = tv.RAYS[findall(bool -> bool, tv.MAXIMAL_CONES[i,:]),:]

pm::Matrix<pm::Rational>
0 -1
-1 1/2


In [95]:
function slicematrix(A)
    return [A[i, :] for i in 1:size(A,1)]
end

function rowMinors(A,S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

function convertIncidenceMatrix(A)
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end

function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end

function findBarycenter(s,X)
    rays = rowMinors(Array(X.RAYS), s)
    dim=size(rays,2)
    bary=zeros(Polymake.Rational,dim,1)
    for i in 1:size(rays,1)
        bary+=rays[i,:]
    end
    bary=Polymake.common.primitive(bary)
    return bary
end

function toric_blowup(s, X, v)
    s = [i + 1 for i in s]
    if v==nothing
        v=findBarycenter(s,X)
    end
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = [coneList[i] for i in starIndex]
    rayMatrix = X.RAYS
    
    lattice = X.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
        rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
        for cone in rank_c_subcones
            new_cone = [i+1 for i in cone]
            if all((i -> i in t).(new_cone))
                push!(clStar, new_cone)
            end
        end
    end
    clStar = unique(clStar)
    
    n = size(rayMatrix, 1) + 1
    coneList = filter(x -> !(x in star), coneList)
    
    if length(s) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                push!(newCones, sort(push!(t, s[1])))
            end
        end
        finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
        return Polymake.fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            push!(newCones, push!(t, n))
        end
    end
    finalRays = vcat((X.RAYS),transpose(v))
    finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
    print(finalCones)
    return Polymake.fulton.NormalToricVariety(INPUT_RAYS = finalRays, INPUT_CONES = finalCones)
end

function convertBool(B)
    out=[]
    for i in 1:size(B,1)
        if B[i]==true
           append!(out,i-1) 
        end
    end
    return out
end

convertBool (generic function with 1 method)

In [96]:
function getConeRays(coneRayIndices, rayMatrix)
    return convert(Matrix{Int}, tv.RAYS[findall(bool -> bool, coneRayIndices),:])
end

getConeRays (generic function with 1 method)

In [97]:
function getConeRank(coneRayIndices, rayMatrix)
    coneRays = rowMinors(rayMatrix,coneRayIndices)
    return rank(Matrix(coneRays))
end

getConeRank (generic function with 1 method)

In [98]:
function getDimension(X)
    return size(X.RAYS, 2)
end

getDimension (generic function with 1 method)

In [99]:
getDimension(tv)

2

In [100]:
# Old function, need to replace with new method within the code
function getConeFaces(coneRayIndices, rayMatrix)
    outCones = []
    # We want to find cones with dimension 1 less than the original
    # Calculate dimension of cone - 1
    c = getConeRank(coneRayIndices, rayMatrix) - 1
    for i in 1:size(coneRayIndices)
        # Iterate through the cone rays in order to iterate through
        # all subcones that are missing one ray from the original
        subcone = deleteat!(copy(coneRayIndices), i)
        
        # Check that it has dimension 1 less than the original
        if (getConeRank(subcone, rayMatrix) == c)
            push!(outCones, subcone)
        end
    end
    # for subcone in powerset(coneRayIndices, min=1, max=length(a)-1)
        # if (getConeRank(subcone, rayMatrix) == c)
            # push!(outCones, subcone)
    # end
    
    return outCones
    # This code may be incorrect, since I am not sure whether we
    # need to consider subcones missing more than 1 ray
end

getConeFaces (generic function with 1 method)

In [101]:


function getConeFacesImproved(fan,cone,rayMatrix)
    lattice = fan.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    cone_faces=[]
    c = rank(Array(rowMinors(rayMatrix, cone))) - 1
    rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
    rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
    for subcone in rank_c_subcones
        new_cone = [i+1 for i in subcone]
        if all((i -> i in cone).(new_cone))
            push!(cone_faces, new_cone)
        end
    end 
    return cone_faces
end

getConeFacesImproved (generic function with 1 method)

In [108]:
function makeSimplicial(X)
    Y = copy(X)
    while (true)
        if Y.SIMPLICIAL==true
            break
        end
        #Maximal cones and ray matrix
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        badCone = nothing
        for i in 1:size(coneList,1)
            cone = coneList[i,:]
            if (getConeRank(cone, rayMatrix) != size(cone))
                badCone = cone
            end
        end
        if (badCone == nothing)
            # All cones are linearly independent
            break
        else
            # Find the first ray that is contained in more than one orbit
            # and subdivide at that ray, using toricBlowup
            
            # Get faces (need to replace this)
            edges = getConeFacesImproved(Y,badCone,rayMatrix)
            # Find the first ray that is contained in more than one orbit
            i = 1
            while count(r->(badCone[i] in r), edges) == 1
                i += 1
            end
            # Subdivide at the cone containing just that ray
            Y = toric_blowup(convertBool(badCone), Y,nothing)
            #Y = toric_blowup([badCone[i]], Y,nothing)
        end
        # Repeat this process until there are no more bad cones
        #break
    end
    return Y
end

makeSimplicial (generic function with 1 method)

In [14]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0 0;0 1 0;2 1 1; 1 2 1],INPUT_CONES=[[0,1,2,3]])
X.MAXIMAL_CONES
Y=copy(X)
Y.MAXIMAL_CONES

pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}


In [23]:
X.SIMPLICIAL

false

In [109]:
makeSimplicial(X)


[[1, 3, 4], [0, 1, 4], [0, 2, 4], [2, 3, 4]]

type: NormalToricVariety

COMBINATORIAL_DIM
	2

FAN_AMBIENT_DIM
	3

INPUT_CONES
	{1 3 4}
	{0 1 4}
	{0 2 4}
	{2 3 4}

INPUT_RAYS
	1 0 0
	0 1 0
	1 1/2 1/2
	1 2 1
	1 1 1

LINEALITY_DIM
	0

LINEALITY_SPACE
	

MAXIMAL_CONES
	{0 1 2}
	{1 3 4}

MAXIMAL_CONES_COMBINATORIAL_DIMS
	2 2

N_MAXIMAL_CONES
	2

PURE
	true

RAYS
	0 1 0
	1 1 1
	1 0 0
	1 1/2 1/2
	1 2 1

SIMPLICIAL
	true

In [85]:
Z.SIMPLICIAL

true